In [1]:
import nltk
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
#import data
train_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/train.csv"
test_data="/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/dev.csv"
train=pd.read_csv(train_data)
train_label=train.meanGrade
test_label=train_label[9100:]
train_label=train_label[:9100]
train_news=train.original
funny_word=train.edit
train_id=train.id

In [3]:
def find_tag(news):
    p = re.compile(r'[<](.*?)/[>]', re.S)
    return re.findall(p, news)
print(train_news[0])

France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq


In [4]:
#do preprocess
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
tt=nltk.tokenize.regexp.WordPunctTokenizer()
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
def preprocess_news(sentence):
    final=[]
    num=0
    for event in sentence:
        BOW={}
        ori=event
        event=removePunctuation(event)
        event=tt.tokenize(event)
        for words in event:
            words=lemmatizer.lemmatize(words)
            if words not in stopwords:
                BOW[words.lower()]=BOW.get(words,0)+1
        BOW[funny_word[num].lower()]=BOW.get(funny_word[num],0)+10
        final.append(BOW)
        num+=1
    return final

punctuation = '!,;:?"\'.\'/<>'
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip()

train_news=preprocess_news(train_news)
test_news=train_news[9100:]
train_news=train_news[:9100]

In [5]:
print(len(train_news))
print(len(test_news))
print(len(train_label))
print(len(test_label))
print(train_news[0])

9100
552
9100
552
{'france': 1, '‘': 1, 'hunting': 1, 'citizen': 1, 'joined': 1, 'isis': 1, '’': 1, 'without': 1, 'trial': 1, 'iraq': 1, 'twins': 10}


In [30]:
#transform to vector
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer=DictVectorizer()
transformer=TfidfTransformer(smooth_idf=False,norm=None)
train_matrix=vectorizer.fit_transform(train_news)
train_matrix=transformer.fit_transform(train_matrix)
test_matrix=vectorizer.transform(test_news)
test_matrix=transformer.transform(test_matrix)
print(test_matrix.shape)
print(train_matrix[0].shape)

(552, 10191)
(1, 10191)


In [268]:
from sklearn.linear_model import LinearRegression
model=LinearRegression(fit_intercept=True, normalize=False, copy_X=False, n_jobs=None)
model.fit(train_matrix,train_label)
pre1=model.predict(test_matrix)
pre=[]
for i in pre1:
    if i<0:
        i=0
    elif i>1:
        i=1
    pre.append(i)

In [7]:
from sklearn import svm
model2=svm.SVR(kernel='rbf')
model2.fit(train_matrix,train_label)
pre2=model2.predict(test_matrix)
pre=[]
for i in pre2:
    if i<0:
        i=0
    elif i>1:
        i=1
    pre.append(i)

In [8]:
rmse = np.sqrt(np.mean((test_label - pre)**2))
print(rmse)

0.5805943855462934


In [ ]:
from sklearn.model_selection import train_test_split,KFold,ShuffleSplit,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import svm
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
clf=svm.SVR(kernel='rbf')
score1=cross_val_score(clf, train_matrix, train_label, cv=cv,scoring='r2')
print(score1.mean())
clf2=model=LinearRegression(fit_intercept=True, normalize=False, copy_X=False, n_jobs=None)
score2=cross_val_score(clf2, train_matrix, train_label, cv=cv,scoring='r2')
score2.mean()